# Sistema de Informação de Agravos de Notificação (SINAN)

Esse notebook tem como objetivo realizar o ELT dos dados provenientes do SINAN que serão utilizados no projeto Dengue.

### Libs

In [2]:
%run ../config/bootstrap.py

In [3]:
import pandas as pd
from datetime import datetime 
from utils import get_project_root, save_parquet_in_chunks  
import basedosdados as bd

/home/max/ML/TrabalhoPraticoML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
project_root = get_project_root() 
billing_id = 'trabalho-pratico-ml'

Para fazer login:
``` shell
gcloud auth application-default login
```

In [5]:
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaylimit = 100
%sql bigquery://genuine-orb-470122-v0
# %sql bigquery://trabalho-pratico-ml

# ⌛ Staging
 

In [5]:
%%sql
SELECT * FROM `basedosdados.br_ms_sinan.dicionario` limit 10

 * bigquery://genuine-orb-470122-v0
10 rows affected.


,id_tabela,nome_coluna,chave,cobertura_temporal,valor
0,microdados_dengue,escolaridade_paciente,1,(1),1ª a 4ª série incompleta do EF
1,microdados_dengue,escolaridade_paciente,2,(1),4ª série completa do EF ( antigo 1° grau)
2,microdados_dengue,escolaridade_paciente,3,(1),5ª à 8ª série incompleta do EF (antigo ginásio ou 1° grau)
3,microdados_dengue,raca_cor_paciente,3,(1),Amarela
4,microdados_dengue,idade_paciente,4,(1),Ano
5,microdados_dengue,raca_cor_paciente,1,(1),Branca
6,microdados_dengue,criterio_confirmacao,2,(1),Clínico Epidemiológico
7,microdados_dengue,evolucao_caso,1,(1),Cura
8,microdados_dengue,idade_paciente,2,(1),Dia
9,microdados_dengue,escolaridade_paciente,8,(1),Educação superior completa


# Notifications

In [6]:
%%sql  
SELECT ano, count(*) as qt FROM `basedosdados.br_ms_sinan.microdados_dengue` group by 1 order by 1 

 * bigquery://genuine-orb-470122-v0
26 rows affected.


,ano,qt
0,2000,172855
1,2001,488590
2,2002,897093
3,2003,416609
4,2004,136867
5,2005,261501
6,2006,411022
7,2007,717097
8,2008,919324
9,2009,600658


# bi_query

### all classes (mainly low_risk)

In [6]:
%%sql df_22 <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where ano = 2022

 * bigquery://genuine-orb-470122-v0
1393877 rows affected.
Returning data to local variable df_22


In [8]:
df_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393877 entries, 0 to 1393876
Data columns (total 47 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   sigla_uf_residencia            1393877 non-null  object 
 1   idade_paciente                 1393877 non-null  object 
 2   sexo_paciente                  1393852 non-null  object 
 3   raca_cor_paciente              1393845 non-null  object 
 4   gestante_paciente              1393486 non-null  object 
 5   possui_doenca_autoimune        1337655 non-null  object 
 6   possui_diabetes                1337655 non-null  object 
 7   possui_doencas_hematologicas   1337655 non-null  object 
 8   possui_hepatopatias            1337655 non-null  object 
 9   possui_doenca_renal            1337655 non-null  object 
 10  possui_hipertensao             1337655 non-null  object 
 11  possui_doenca_acido_peptica    1337655 non-null  object 
 12  paciente_vacin

In [ ]:
save_parquet_in_chunks(df_22, "data/silver/2022_all_classes")

Salvando arquivos Parquet: 100%|██████████| 1393877/1393877 [02:29<00:00, 9307.95linhas/s]

✔️ 2022_part_1.parquet salvo com 7.33 MB (1393877 linhas)
✅ Todos os arquivos salvos com sucesso.


In [11]:
del(df_22)

## alarm

In [ ]:
%%sql df_alarm <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where classificacao_final = '11' and evolucao_caso != '2'
and ano in (2020, 2021, 2023, 2024, 2025) -- Sem 2022 para não repetir dados

 * bigquery://genuine-orb-470122-v0
186298 rows affected.
Returning data to local variable df_alarm


In [ ]:
save_parquet_in_chunks(df_alarm, "data/silver/2020-2025_alarm", 50, compression='brotli')

Salvando arquivos Parquet: 100%|██████████| 186298/186298 [00:25<00:00, 7216.59linhas/s]

✔️ 2020-2025_alarm_part_1.parquet salvo com 0.66 MB (186298 linhas)
✅ Todos os arquivos salvos com sucesso.


In [16]:
del(df_alarm)

## severe

In [ ]:
%%sql df_severe <<
SELECT 
        sigla_uf_residencia
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        ,apresenta_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,prova_laco
        ,classificacao_final
        ,evolucao_caso
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as dias_sintomas_notificacao

from `basedosdados.br_ms_sinan.microdados_dengue`
where (classificacao_final = '12' or evolucao_caso = '2') -- Casos de Dengue Grave ou Evolução para óbito
and ano in (2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
            2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
            2020, 2021, 2023, 2024, 2025) -- Sem 2022 para não repetir dados

 * bigquery://genuine-orb-470122-v0
15223 rows affected.
Returning data to local variable df_severe


In [ ]:
save_parquet_in_chunks(df_severe, "data/silver/2000-2025_severe", 50, compression='brotli')

Salvando arquivos Parquet: 100%|██████████| 15223/15223 [00:01<00:00, 14768.18linhas/s]

✔️ 2000-2025_severe_part_1.parquet salvo com 0.09 MB (15223 linhas)
✅ Todos os arquivos salvos com sucesso.


In [17]:
del(df_severe)